# Lab 4
The goals of this lab are as follows:
1. introduce criticality calculations; and
2. compare OpenMC criticality calculation for a simple geometry and material composition to 1-Group Critical Equation.

The problem is to find the critical radius of a solid sphere of pure Pu-239.  


## 1-Group Critical Equation
The analytic solution is arrived at by using the 1-Group Critical Equation given by:
$$B_1^2 = \frac{k_{\infty}-1}{L^2}$$
where $B_1^2$ is the buckling, and $L^2$ is the diffusion area; and $k_{\infty}$ is the infinite multiplication factor.  

The buckling depends on geometry; for this problem, the domain is spherical and buckling is given by:
$$B^{2}=\left( \frac{\pi}{\tilde{R}}\right)^{2}$$
where $\tilde{R}$ is the extrapolated radius of the sphere: $\tilde{R}=R+d$; with $d$ being the extrapolation distance.  The extrapolation distance is base on the diffusion coefficient: $d = 2.13D$ and the diffusion coefficient $(D)$ is based on the macroscopic transport cross section: $D=\frac{1}{3\Sigma_{tr}}$.

The macroscopic transport cross section is, in turn, computed from the corresponding microscopic cross section $(\sigma_{tr})$ and the atom density of the Pu-239 material.  

We're almost to the bottom of the rabbit hole...

The diffusion area is based on the diffusion coefficient and the macroscopic absorption cross section:
$$L^{2}=\frac{D}{\Sigma_{a}}$$
and, similar to the macroscopic transport cross section, $\Sigma_{a}$ is computed with the atom density of the Pu-239 material and the corresponding microscopic cross section $(\sigma_a)$

Lastly, $k_{\infty}$ is the infinite multiplication factor.  It is based on the regeneration factor ($\eta$) which we will take to be a known material property of Pu-239 and the fuel utilization, $f$, which we will take to be unity since the domain where neutrons will be diffusing and interacting is 100% fuel.

All of this long discussion is implemented in the code below.  Note that all elements of the equations above can be taken to be known *except* for the radius $R$.


In [2]:
import numpy as np

# data:
N_Pu = 0.037; #atoms/b-cm, atom density of Pu-239
sig_f = 1.85; #b, fission micro xc
sig_a = 2.11; #b, absorption micro xc
sig_tr = 6.8; #b, transport micro xc

Sig_tr = N_Pu*sig_tr; # 1/cm, transport macro xc
D = 1./(3.*Sig_tr); #cm, diffusion coefficient
d = 2.13*D; #cm, extrapolation length

f = 1.; # fuel utilization
eta_pu = 2.61; # 1-group reproduction factor for Pu-239

k_inf = f*eta_pu;

Sig_a = N_Pu*sig_a; # 1/cm, abs macro xc
L_sq = D/Sig_a; # cm**2, diffusion area
B_sq = (k_inf-1.)/L_sq; #1/cm**2, buckling

R_tild = np.pi/(np.sqrt(B_sq)); # cm, extrapolated radius

R = R_tild - d; # cm, 1-group critical radius

print(f'1-group critical radius for sphere of Pu-238: %5.4f cm'%R)






1-group critical radius for sphere of Pu-238: 7.3776 cm


## OpenMC model
Below we will implment this simple model and use OpenMC features to find the critical radius.

In [3]:
import openmc

In order to do this analysis we will use OpenMC's criticality search machinery.  The basic idea is that we need to create a function that takes one parameter - in this case $R$ - that will be varied while we search for the value to that parameter that results in $k_{eff}$ being equal to one.

In [9]:
# create the model.  "R" will be the parametric variable
def build_model(R):
    # create the sphere material
    fuel = openmc.Material(name='fuel');
    fuel.add_nuclide('Pu239',1);
    fuel.set_density('atom/b-cm',N_Pu);

    materials = openmc.Materials([fuel]); 
    
    fuel_sphere = openmc.Sphere(r=R,boundary_type='vacuum');
    
    fuel_cell = openmc.Cell();
    fuel_cell.fill = fuel;
    fuel_cell.region = -fuel_sphere;
    
    root_universe = openmc.Universe();
    root_universe.add_cells([fuel_cell]);
    
    geometry = openmc.Geometry(root_universe);
    
    settings = openmc.Settings()
    settings.batches = 200;
    settings.inactive = 50;
    settings.particles = 10000;
    
    bounds = [-R,-R,-R,R,R,R];
    uniform_dist = openmc.stats.Box(bounds[:3],bounds[3:],
                                    only_fissionable=True);
    settings.source = openmc.source.Source(space=uniform_dist);
    
    # so we don't waste disk input/output tallies that we will  not use
    settings.output = {'tallies':False};
    
    model = openmc.model.Model(geometry,materials,settings);
    
    return model

### Search for Critical Radius
To perform the search we will employ a call to 
<code> openmc.search_for_keff</code> function and pass in the relevant arguments.

In [11]:
crit_R, guesses, keffs = openmc.search_for_keff(build_model,
                                               bracket=[4.,15.],
                                               tol=1e-3,print_iterations=True)

print(f'Critical Radius: %5.4f'%crit_R);

Iteration: 1; Guess of 4.00e+00 produced a keff of 0.63274 +/- 0.00029
Iteration: 2; Guess of 1.50e+01 produced a keff of 1.82488 +/- 0.00090
Iteration: 3; Guess of 9.50e+00 produced a keff of 1.33705 +/- 0.00071
Iteration: 4; Guess of 6.75e+00 produced a keff of 1.01102 +/- 0.00051
Iteration: 5; Guess of 5.38e+00 produced a keff of 0.82847 +/- 0.00041
Iteration: 6; Guess of 6.06e+00 produced a keff of 0.92161 +/- 0.00045
Iteration: 7; Guess of 6.41e+00 produced a keff of 0.96777 +/- 0.00052
Iteration: 8; Guess of 6.58e+00 produced a keff of 0.98935 +/- 0.00051
Iteration: 9; Guess of 6.66e+00 produced a keff of 1.00126 +/- 0.00053
Iteration: 10; Guess of 6.62e+00 produced a keff of 0.99563 +/- 0.00050
Iteration: 11; Guess of 6.64e+00 produced a keff of 0.99761 +/- 0.00054
Iteration: 12; Guess of 6.65e+00 produced a keff of 0.99904 +/- 0.00047
Iteration: 13; Guess of 6.66e+00 produced a keff of 1.00004 +/- 0.00054
Critical Radius: 6.6587
